In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install py7zr
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 92.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.16
    Uninstalling botocore-1.35.16:
      Successfully uninstalled botocore-1.35.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.0 requires botocore<1.35.17,>=1.35.16, but you have botocore 1.29.165 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstall

In [34]:
from tqdm import tqdm

In [3]:
from datasets import load_dataset

In [4]:
squad = load_dataset('squad')
squad

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [5]:
squad['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [7]:
def preprocess(batch):
    return {
        'input_text': [f"question: {q} context: {c}" for q, c in zip(batch['question'], batch['context'])],
        'target_text': [" ".join(ans['text']) if ans['text'] else "" for ans in batch['answers']]
    }

tokenized_squad = squad.map(preprocess, batched=True)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [8]:
tokenized_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text'],
        num_rows: 10570
    })
})

In [9]:
tokenized_squad['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]},
 'input_text': 'question: To whom did the Virgin M

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset):

    start_idx = 0
    end_idx = 0

    if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
        pass
        # print("target is (0, 0)")
        # nothing else to do

    else:
        # find the start & end token position

        i = ctx_start
        for start_end_char in offset[ctx_start:]:
            start, end = start_end_char

            if start == ans_start_char:
                start_idx = i

            if end == ans_end_char:
                end_idx = i
                break

            i += 1

    return start_idx, end_idx

In [12]:
max_length = 384
stride = 128

def tokenizer_fn_train(batch):
    questions = [q.strip() for q in batch['question']]

    inputs = tokenizer(questions,
                     batch['context'],
                     max_length = max_length,
                     stride = stride,
                     truncation = 'only_second',
                     return_overflowing_tokens = True,
                     return_offsets_mapping = True,
                     padding = 'max_length')

    offset_mapping = inputs.pop('offset_mapping')
    orig_sample_idxs = inputs.pop('overflow_to_sample_mapping')
    answers = batch['answers']
    start_idxs, end_idxs = [], []

    for i, offset in enumerate(offset_mapping):
        sample_idx = orig_sample_idxs[i]
        answer = answers[sample_idx]

        ans_start_char = answer['answer_start'][0]
        ans_end_char = ans_start_char + len(answer['text'][0])

        sequence_ids = inputs.sequence_ids(i)

        ctx_start = sequence_ids.index(1)
        ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

        start_idx, end_idx = find_answer_token_idx(
        ctx_start,
        ctx_end,
        ans_start_char,
        ans_end_char,
        offset
        )

        start_idxs.append(start_idx)
        end_idxs.append(end_idx)

    inputs['start_positions'] = start_idxs
    inputs['end_positions'] = end_idxs

    return inputs

In [13]:
train_dataset = squad['train'].map(
    tokenizer_fn_train,
    batched = True,
    remove_columns = squad['train'].column_names
)

len(squad['train']), len(train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 88729)

In [14]:
def tokenize_fn_validation(batch):

    questions = [q.strip() for q in batch['question']]

    inputs = tokenizer(questions,
                     batch['context'],
                     max_length = max_length,
                     stride = stride,
                     truncation = 'only_second',
                     return_overflowing_tokens = True,
                     return_offsets_mapping = True,
                     padding = 'max_length')

    orig_sample_idxs = inputs['overflow_to_sample_mapping']
    sample_ids = []

    for i in range(len(inputs['input_ids'])):
        sample_idx = orig_sample_idxs[i]
        sample_ids.append(batch['id'][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs['offset_mapping'][i]
        inputs['offset_mapping'][i] = [x if sequence_ids[j] == 1 else None for j, x in enumerate(offset)]

    inputs['sample_id'] = sample_ids
    return inputs

In [15]:
validation_dataset = squad['validation'].map(
    tokenize_fn_validation,
    batched = True,
    remove_columns = squad['validation'].column_names
)

len(squad['validation']), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10822)

In [43]:
n_largest = 20
max_answer_length = 30

In [37]:
def compute_metrics(start_logits, end_logits, processed_dataset, orig_dataset):
  sample_id2idxs = {}

  for i, id_ in enumerate(processed_dataset['sample_id']):
    if id_ not in sample_id2idxs:
      sample_id2idxs[id_] = [i]

    else:
      sample_id2idxs[id_].append(i)

  predicted_answers = []
  for sample in tqdm(orig_dataset):
    sample_id = sample['id']
    context = sample['context']

    best_score = float('-inf')
    best_answer = None

    for idx in sample_id2idxs[sample_id]:
      start_logit = start_logits[idx]
      end_logit = end_logits[idx]

      offsets = processed_dataset[idx]['offset_mapping']

      start_indices = (-start_logit).argsort()
      end_indices = (-end_logit).argsort()

      for start_idx in start_indices[:n_largest]:
        for end_idx in end_indices[:n_largest]:

          if offsets[start_idx] is None or offsets[end_idx] is None:
            continue

          if start_idx > end_idx:
            continue

          if end_idx - start_idx + 1 > max_answer_length:
            continue

          score = start_logit[start_idx] + end_logit[end_idx]

          if score > best_score:
            best_score = score

            first_ch = offsets[start_idx][0]
            end_ch = offsets[end_idx][1]

            best_answer = context[first_ch : end_ch]

    predicted_answers.append({'id' : sample_id, 'prediction_text' : best_answer})
  true_answers = [{'id' : x['id'], 'answers' : x['answers']} for x in orig_dataset]

  return metric.compute(predictions = predicted_answers, references = true_answers)

In [38]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

args = TrainingArguments(
    'finedtuned-squad',
    eval_strategy = 'no',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    weight_decay = 0.01,
    num_train_epochs = 3,
    fp16 = True
)

In [39]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset = train_dataset.shuffle(seed = 42).select(range(10_000)),
    eval_dataset = validation_dataset,
    tokenizer = tokenizer
)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [41]:
trainer.train()

Step,Training Loss
500,1.395600
1000,1.331700
1500,1.114100
2000,0.786700
2500,0.775700
3000,0.498200
3500,0.446400


TrainOutput(global_step=3750, training_loss=0.8775292846679688, metrics={'train_runtime': 1224.8283, 'train_samples_per_second': 24.493, 'train_steps_per_second': 3.062, 'total_flos': 5879177026560000.0, 'train_loss': 0.8775292846679688, 'epoch': 3.0})

In [44]:
outputs = trainer.predict(validation_dataset)

predictions, _, _ = outputs

start_logits, end_logits = predictions

compute_metrics(start_logits,
                end_logits,
                validation_dataset,
                squad['validation'])

100%|██████████| 10570/10570 [00:24<00:00, 424.60it/s]


{'exact_match': 70.47303689687796, 'f1': 80.53739595254949}

In [46]:
trainer.save_model('distil_bert')

In [ ]:
trainer.evaluate(tokenized_squad['validation'])

In [47]:
from transformers import pipeline

pipe = pipeline('question-answering', model = 'distil_bert', device = 0)

In [48]:
squad['train'][0]['question']

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [49]:
squad['train'][0]['answers']

{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}

In [54]:
context = """
The Amazon rainforest, often referred to as the "lungs of the Earth," is the largest tropical rainforest in the world, spanning across several countries, including Brazil, Peru, and Colombia. It is home to an incredibly diverse array of wildlife, with millions of species of plants, animals, and insects. The rainforest plays a crucial role in regulating the global climate by absorbing carbon dioxide and producing oxygen. However, deforestation and illegal logging pose significant threats to this vital ecosystem, leading to habitat loss and contributing to climate change.
"""

question = "What is one of the major threats to the Amazon rainforest mentioned in the paragraph"

In [58]:
pipe(question, context)

{'score': 0.7816429734230042,
 'start': 434,
 'end': 467,
 'answer': 'deforestation and illegal logging'}

In [57]:
!zip distilbert.zip -r distil_bert/

  adding: distil_bert/ (stored 0%)
  adding: distil_bert/tokenizer_config.json (deflated 76%)
  adding: distil_bert/special_tokens_map.json (deflated 42%)
  adding: distil_bert/training_args.bin (deflated 51%)
  adding: distil_bert/config.json (deflated 48%)
  adding: distil_bert/tokenizer.json (deflated 70%)
  adding: distil_bert/model.safetensors (deflated 7%)
  adding: distil_bert/vocab.txt (deflated 49%)
